In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_DB = os.environ.get("POSTGRES_DB")
POSTGRES_HOST = os.environ.get("POSTGRES_HOST")
POSTGRES_PORT = os.environ.get("POSTGRES_PORT")

In [2]:
conn = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(conn)

In [4]:
query = "Select * from raw_data;"

df = pd.read_sql_query(query, conn)

df.describe()

,id,magnitude,latitude,longitude,depth,utc_time
count,179385.000000,179385.000000,179385.000000,179385.000000,179385.00000,179385
mean,89693.000000,2.154831,39.235059,-107.419263,34.07729,2023-06-10 14:57:53.423674624
min,1.000000,1.000000,-65.849700,-179.998700,-3.74000,2022-08-07 01:56:38
25%,44847.000000,1.300000,31.618907,-153.202100,5.51000,2023-01-02 00:50:12
50%,89693.000000,1.800000,39.178000,-137.763600,10.00000,2023-06-03 08:02:10
75%,134539.000000,2.500000,59.793300,-110.291000,35.00000,2023-11-14 07:35:31
max,179385.000000,7.800000,86.593900,179.999400,681.24000,2024-05-01 23:48:51
std,51784.133357,1.138389,23.116535,84.521006,65.77905,NaN
